*This notebook serves as an example of how to run the FinAI evaluation harness to benchmark models across various financial datasets.*

**Important: When you run, make sure you choose runtime A100 GPU (Google Colab)**

Clone the repo and install required dependencies

In [1]:
!git clone https://github.com/The-FinAI/PIXIU --recursive
%cd PIXIU
!pip install -r requirements.txt
%cd /content/PIXIU/src/financial-evaluation
!pip install -e .[multilingual]
!pip install bert_score
!pip install gdown
!pip install vllm==0.5.4
!pip install torch==2.4.0 torchvision==0.19
!pip install peft

fatal: destination path 'PIXIU' already exists and is not an empty directory.
/content/PIXIU
Processing ./src/factscore_package/en_core_web_sm-3.7.1.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for en_core_web_sm: filename=en_core_web_sm-3.7.1-py3-none-any.whl size=12803373 sha256=dca44a62aa0d8de1166dd6300f3aa129f03bade84d6d218facbed56314eafe2d
  Stored in directory: /root/.cache/pip/wheels/64/e9/df/1c136b2b33655373f405db67954521cfd390f783090c2df8a0
Successfully built en_core_web_sm
  Attempting uninstall: en_core_web_sm
    Found existing installation: en_core_web_sm 3.7.1
    Uninstalling en_core_web_sm-3.7.1:
      Successfully uninstalled en_core_web_sm-3.7.1
/content/PIXIU/src/financial-evaluation
Obtaining file:///content/PIXIU/src/financial-evaluation
  Preparing metadata (setup.py) ... done
  Attempting uninstall: lm_eval
    Found existing installation: lm_eval 0.3.0
    Uninstalling lm_eval-0.3.0:
      Successfully uninstalled lm_eval-0.3.0
  Running setup

Connect to Google Drive and Download BART checkpoint to src/metrics/BARTScore/

In [3]:
from google.colab import drive
import os
import gdown

drive.mount('/content/drive')

source_path = "/content/drive/My Drive/bart_score.pth"
destination_path = "/content/PIXIU/src/metrics/BARTScore/bart_score.pth"

if os.path.exists(source_path) and not os.path.exists(destination_path):
    !cp "{source_path}" "{destination_path}"
    print("File found in Google Drive and copied.")
else:
    file_id = '19Fpob1RhQHyvMlOqxPO89z1W58PvkOm-'
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, destination_path, quiet=False)
    print("File not found in Google Drive. Downloaded instead.")


Mounted at /content/drive


Downloading...
From (original): https://drive.google.com/uc?id=19Fpob1RhQHyvMlOqxPO89z1W58PvkOm-
From (redirected): https://drive.google.com/uc?id=19Fpob1RhQHyvMlOqxPO89z1W58PvkOm-&confirm=t&uuid=48f5e04f-f2cc-42fe-9359-1d726b67e32b
To: /content/PIXIU/src/metrics/BARTScore/bart_score.pth
100%|██████████| 1.63G/1.63G [00:33<00:00, 48.0MB/s]

File not found in Google Drive. Downloaded instead.


In [4]:
%cd /content/PIXIU/src

/content/PIXIU/src


In [5]:
%cd /content

/content


In [6]:
import os

os.environ['PYTHONPATH'] += ":/content/PIXIU/src/metrics/BARTScore/"
!echo $PYTHONPATH

/env/python:/content/PIXIU/src/metrics/BARTScore/


# Login with your Hugging Face Access Token. Make sure to request access to models on Hugging Face before use.

In [8]:
from huggingface_hub import login
#login(token="HUGGINGFACE_TOKEN")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Currently availible datasets
1. **NER**: `flare_ner`
2. **FINER-ORD**: `flare_finer_ord`
3. **FinRED**: `flare_finred`
4. **SC**: `flare_causal20_sc`
5. **CD**: `flare_cd`
6. **FNXL**: `flare_fnxl`
7. **FSRL**: `flare_fsrl`
8. **FPB**: `flare_fpb`
9. **FiQA-SA**: `flare_fiqasa`
10. **TSA**: `flare_tsa`
11. **Headlines**: `flare_headlines`
12. **FOMC**: `flare_fomc`
13. **FinArg-ACC**: `flare_finarg_ecc_auc`
14. **FinArg-ARC**: `flare_finarg_ecc_arc`
15. **MultiFin**: `flare_multifin_en`
16. **MA**: `flare_ma`
17. **MLESG**: `flare_mlesg`
18. **FinQA**: `flare_finqa`
19. **TATQA**: `flare_tatqa`
20. **Regulations**: (No specific task name provided for this)
21. **ConvFinQA**: `flare_convfinqa`
22. **EDTSUM**: `flare_edtsum`
23. **ECTSUM**: `flare_ectsum`
24. **BigData22**: `flare_sm_bigdata`
25. **ACL18**: `flare_sm_acl`
26. **CIKM18**: `flare_sm_cikm`
27. **German**: `flare_german`
28. **Australian**: `flare_australian`
29. **LendingClub**: `flare_cra_lendingclub`
30. **ccf**: `flare_cra_ccf`
31. **ccfraud**: `flare_cra_ccfraud`
32. **polish**: `flare_cra_polish`
33. **taiwan**: `flare_cra_taiwan`
34. **portoseguro**: `flare_cra_portoseguro`
35. **travelinsurance**: `flare_cra_travelinsurace`
36. **ESMultiFin**: `flare_es_multifin`
37. **ESEFP**: `flare_es_efp`
38. **ESEFPA**: `flare_es_efpa`
39. **ESTSA**: `flare_es_tsa`
40. **ESFINANCEES**: `flare_es_financees`
40. **ESFNS**: `flare_es_fns`

# Example Usage

In [9]:
trust_remote_code=True

# meta-llama/Meta-Llama-3.1-8B on ESFNS dataset

In [11]:
!python PIXIU/src/eval.py \
    --model "hf-causal-vllm" \
    --model_args "pretrained=meta-llama/Meta-Llama-3.1-8B,tokenizer=meta-llama/Meta-Llama-3.1-8B,dtype=float16,use_fast=False,max_gen_toks=25" \
    --tasks "flare_es_multifin" \
    --batch_size 20000 \
    --num_fewshot 0

2024-08-30 06:08:11.042735: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-30 06:08:11.058717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-30 06:08:11.080395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-30 06:08:11.086999: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-30 06:08:11.102997: I tensorflow/core/platform/cpu_feature_guar

# Duxiaoman-DI/XuanYuan-6B-Chat on FiQA-SA dataset

In [12]:
!python PIXIU/src/eval.py \
    --model "hf-causal-vllm" \
    --model_args "pretrained=Qwen/Qwen2-7B-Instruct,tokenizer=Qwen/Qwen2-7B-Instruct,dtype=float16,use_fast=False,max_gen_toks=128" \
    --tasks "flare_fiqasa" \
    --batch_size 20000 \
    --num_fewshot 0

2024-08-30 06:10:08.813250: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-30 06:10:08.829560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-30 06:10:08.851320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-30 06:10:08.857775: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-30 06:10:08.873323: I tensorflow/core/platform/cpu_feature_guar